In [1]:
!pip install sentence-transformers pinecone-client

In [2]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import pinecone

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


# 테스트1

In [3]:
df = pd.read_csv('Travel_noquery.csv', index_col=0)
df.head()

,org,dest,days,visiting_city_number,date,people_number,local_constraint,budget,level,annotated_plan,reference_information
0,St. Petersburg,Rockford,3,1,"['2022-03-16', '2022-03-17', '2022-03-18']",1,"{'house rule': None, 'cuisine': None, 'room ty...",1700,easy,"[{'org': 'St. Petersburg', 'dest': 'Rockford',...","[{'Description': 'Attractions in Rockford', 'C..."
1,Kansas City,Pensacola,3,1,"['2022-03-27', '2022-03-28', '2022-03-29']",1,"{'house rule': None, 'cuisine': None, 'room ty...",900,easy,"[{'org': 'Kansas City', 'dest': 'Pensacola', '...","[{'Description': 'Attractions in Pensacola', '..."
2,Las Vegas,Stockton,3,1,"['2022-03-03', '2022-03-04', '2022-03-05']",1,"{'house rule': None, 'cuisine': None, 'room ty...",1400,easy,"[{'org': 'Las Vegas', 'dest': 'Stockton', 'day...","[{'Description': 'Attractions in Stockton', 'C..."
3,New Orleans,Louisville,3,1,"['2022-03-12', '2022-03-13', '2022-03-14']",1,"{'house rule': None, 'cuisine': None, 'room ty...",1900,easy,"[{'org': 'New Orleans', 'dest': 'Louisville', ...","[{'Description': 'Attractions in Louisville', ..."
4,Las Vegas,Denver,3,1,"['2022-03-12', '2022-03-13', '2022-03-14']",1,"{'house rule': None, 'cuisine': None, 'room ty...",1600,easy,"[{'org': 'Las Vegas', 'dest': 'Denver', 'days'...","[{'Description': 'Attractions in Denver', 'Con..."


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45 entries, 0 to 44
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   org                    45 non-null     object
 1   dest                   45 non-null     object
 2   days                   45 non-null     int64 
 3   visiting_city_number   45 non-null     int64 
 4   date                   45 non-null     object
 5   people_number          45 non-null     int64 
 6   local_constraint       45 non-null     object
 7   budget                 45 non-null     int64 
 8   level                  45 non-null     object
 9   annotated_plan         45 non-null     object
 10  reference_information  45 non-null     object
dtypes: int64(4), object(7)
memory usage: 4.2+ KB


In [5]:
# org부터 level까지의 열을 JSON 형식으로 결합하여 새로운 열에 저장
df['combined_plan'] = df.apply(
    lambda row: {
        'org': row['org'],
        'dest': row['dest'],
        'days': row['days'],
        'visiting_city_number': row['visiting_city_number'],
        'date': row['date'],
        'people_number': row['people_number'],
        'local_constraint': row['local_constraint'],
        'budget': row['budget'],
        'level': row['level']
    }, axis=1
)

# 결과 출력
print(df[['combined_plan', 'annotated_plan']])

                                        combined_plan  \
0   {'org': 'St. Petersburg', 'dest': 'Rockford', ...   
1   {'org': 'Kansas City', 'dest': 'Pensacola', 'd...   
2   {'org': 'Las Vegas', 'dest': 'Stockton', 'days...   
3   {'org': 'New Orleans', 'dest': 'Louisville', '...   
4   {'org': 'Las Vegas', 'dest': 'Denver', 'days':...   
5   {'org': 'Orlando', 'dest': 'Texas', 'days': 5,...   
6   {'org': 'Santa Ana', 'dest': 'Montana', 'days'...   
7   {'org': 'Baton Rouge', 'dest': 'Texas', 'days'...   
8   {'org': 'Memphis', 'dest': 'Pennsylvania', 'da...   
9   {'org': 'Greer', 'dest': 'Pennsylvania', 'days...   
10  {'org': 'Houston', 'dest': 'Utah', 'days': 7, ...   
11  {'org': 'Oklahoma City', 'dest': 'Colorado', '...   
12  {'org': 'Gulfport', 'dest': 'Texas', 'days': 7...   
13  {'org': 'Daytona Beach', 'dest': 'Texas', 'day...   
14  {'org': 'Punta Gorda', 'dest': 'Wisconsin', 'd...   
15  {'org': 'Fort Lauderdale', 'dest': 'Norfolk', ...   
16  {'org': 'Sacramento', 'dest

In [6]:
df['combined_plan'] = df['combined_plan'].astype(str)

In [7]:
temp = df['combined_plan'].iloc[0]
type(temp)

str

In [8]:
df.head()

,org,dest,days,visiting_city_number,date,people_number,local_constraint,budget,level,annotated_plan,reference_information,combined_plan
0,St. Petersburg,Rockford,3,1,"['2022-03-16', '2022-03-17', '2022-03-18']",1,"{'house rule': None, 'cuisine': None, 'room ty...",1700,easy,"[{'org': 'St. Petersburg', 'dest': 'Rockford',...","[{'Description': 'Attractions in Rockford', 'C...","{'org': 'St. Petersburg', 'dest': 'Rockford', ..."
1,Kansas City,Pensacola,3,1,"['2022-03-27', '2022-03-28', '2022-03-29']",1,"{'house rule': None, 'cuisine': None, 'room ty...",900,easy,"[{'org': 'Kansas City', 'dest': 'Pensacola', '...","[{'Description': 'Attractions in Pensacola', '...","{'org': 'Kansas City', 'dest': 'Pensacola', 'd..."
2,Las Vegas,Stockton,3,1,"['2022-03-03', '2022-03-04', '2022-03-05']",1,"{'house rule': None, 'cuisine': None, 'room ty...",1400,easy,"[{'org': 'Las Vegas', 'dest': 'Stockton', 'day...","[{'Description': 'Attractions in Stockton', 'C...","{'org': 'Las Vegas', 'dest': 'Stockton', 'days..."
3,New Orleans,Louisville,3,1,"['2022-03-12', '2022-03-13', '2022-03-14']",1,"{'house rule': None, 'cuisine': None, 'room ty...",1900,easy,"[{'org': 'New Orleans', 'dest': 'Louisville', ...","[{'Description': 'Attractions in Louisville', ...","{'org': 'New Orleans', 'dest': 'Louisville', '..."
4,Las Vegas,Denver,3,1,"['2022-03-12', '2022-03-13', '2022-03-14']",1,"{'house rule': None, 'cuisine': None, 'room ty...",1600,easy,"[{'org': 'Las Vegas', 'dest': 'Denver', 'days'...","[{'Description': 'Attractions in Denver', 'Con...","{'org': 'Las Vegas', 'dest': 'Denver', 'days':..."


In [9]:
# 1. 모델 로드 (Sentence-BERT)
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# 2. 임베딩 생성할 열들
cols_to_embed = ['combined_plan', 'reference_information']

# 각 열에 대해 벡터 임베딩 생성
for col in cols_to_embed:
    df[f'{col}_embeddings'] = df[col].apply(lambda x: model.encode(x))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
model.get_sentence_embedding_dimension()

384

In [11]:
df.head()

,org,dest,days,visiting_city_number,date,people_number,local_constraint,budget,level,annotated_plan,reference_information,combined_plan,combined_plan_embeddings,reference_information_embeddings
0,St. Petersburg,Rockford,3,1,"['2022-03-16', '2022-03-17', '2022-03-18']",1,"{'house rule': None, 'cuisine': None, 'room ty...",1700,easy,"[{'org': 'St. Petersburg', 'dest': 'Rockford',...","[{'Description': 'Attractions in Rockford', 'C...","{'org': 'St. Petersburg', 'dest': 'Rockford', ...","[0.6162878, -0.06970939, 0.03442905, -0.013942...","[-0.0577623, -0.08848809, 0.0641853, 0.0149085..."
1,Kansas City,Pensacola,3,1,"['2022-03-27', '2022-03-28', '2022-03-29']",1,"{'house rule': None, 'cuisine': None, 'room ty...",900,easy,"[{'org': 'Kansas City', 'dest': 'Pensacola', '...","[{'Description': 'Attractions in Pensacola', '...","{'org': 'Kansas City', 'dest': 'Pensacola', 'd...","[0.54303694, -0.0006052768, 0.0073541943, 0.02...","[-0.31180072, 0.07702538, -0.36028045, 0.44311..."
2,Las Vegas,Stockton,3,1,"['2022-03-03', '2022-03-04', '2022-03-05']",1,"{'house rule': None, 'cuisine': None, 'room ty...",1400,easy,"[{'org': 'Las Vegas', 'dest': 'Stockton', 'day...","[{'Description': 'Attractions in Stockton', 'C...","{'org': 'Las Vegas', 'dest': 'Stockton', 'days...","[0.61243695, -0.012746161, 0.05304588, 0.10722...","[0.28747538, -0.2022571, -0.188432, 0.30165887..."
3,New Orleans,Louisville,3,1,"['2022-03-12', '2022-03-13', '2022-03-14']",1,"{'house rule': None, 'cuisine': None, 'room ty...",1900,easy,"[{'org': 'New Orleans', 'dest': 'Louisville', ...","[{'Description': 'Attractions in Louisville', ...","{'org': 'New Orleans', 'dest': 'Louisville', '...","[0.6254805, 0.050417706, 0.21641679, -0.001256...","[-0.23784271, -0.03650164, 0.35464713, 0.17111..."
4,Las Vegas,Denver,3,1,"['2022-03-12', '2022-03-13', '2022-03-14']",1,"{'house rule': None, 'cuisine': None, 'room ty...",1600,easy,"[{'org': 'Las Vegas', 'dest': 'Denver', 'days'...","[{'Description': 'Attractions in Denver', 'Con...","{'org': 'Las Vegas', 'dest': 'Denver', 'days':...","[0.59917665, 0.04538342, 0.06974125, 0.1215144...","[0.30627164, 0.024656627, 0.0114385225, 0.2737..."


In [12]:
# 3. Pinecone 초기화
from pinecone import Pinecone, ServerlessSpec

pinecone = Pinecone(api_key="YOUR-API-KEY")

# 인덱스 생성
index_name = 'travel-index'
if index_name not in pinecone.list_indexes():
    pinecone.create_index(
        name=index_name,
        dimension=model.get_sentence_embedding_dimension(),   # 임베딩 차원은 임베딩 모델 따라 결정됨
        metric='cosine',
        spec=ServerlessSpec(
           cloud="aws",
           region="us-east-1")
    )

In [13]:
# 인덱스 연결
index = pinecone.Index(index_name)

In [14]:
# 4. 벡터 데이터 Pinecone에 업로드
vectors = []
for i, row in df.iterrows():
    # annotated_plan, reference_information 벡터를 모두 합친 후 저장
    combined_vector = row['combined_plan_embeddings'] + row['reference_information_embeddings']
    vectors.append((str(i), combined_vector))

# 벡터 데이터 Pinecone에 업로드
index.upsert(vectors)


{'upserted_count': 45}

In [15]:
# Pinecone 상태 확인
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [18]:
# 쿼리 벡터 생성
query = "Could you assist in creating a travel plan for one person, starting from Houston and heading to Utah for a week? The visit will be from the 23rd of March to the 29th of March, 2022, and will cover 3 different cities in Utah. The budget for this trip is set at $7,700."
query_embedding = model.encode(query).tolist()

# 유사한 벡터 검색 (top_k=5로 상위 5개 결과)
search_results = index.query(vector=query_embedding, top_k=5)

# 검색 결과 출력(유사도 점수 높은 순으로 출력)
for match in search_results['matches']:
    vector_id = int(match['id'])  # 벡터의 ID를 정수로 변환
    print(f"Vector ID: {vector_id}")
    print(f"Score: {match['score']}")
    print(df.iloc[vector_id],"\n")  # 해당 ID에 대응하는 데이터프레임의 행을 출력


Vector ID: 10
Score: 0.521224797
org                                                                           Houston
dest                                                                             Utah
days                                                                                7
visiting_city_number                                                                3
date                                ['2022-03-23', '2022-03-24', '2022-03-25', '20...
people_number                                                                       1
local_constraint                    {'house rule': None, 'cuisine': None, 'room ty...
budget                                                                           7700
level                                                                            easy
annotated_plan                      [{'org': 'Houston', 'dest': 'Utah', 'days': 7,...
reference_information               [{'Description': 'Attractions in Salt Lake Cit...
combined_plan        

---
# 테스트2: 다른 데이터로 테스트

In [19]:
# 각 파일을 데이터프레임으로 읽기
df_instagram = pd.read_csv('instagram_posts_example.csv')
df_naver_blog = pd.read_csv('naver_blog_posts_example.csv')
df_places = pd.read_csv('places_data_example.csv')

In [20]:
df_instagram

,post_id,hashtag,tagged_location,caption,timestamp
0,1,#travel #paris #explore,Eiffel Tower,Exploring the Eiffel Tower today! #paris #travel,2024-10-01 12:34:56
1,2,#art #louvre #museum,Louvre Museum,Amazing art collection at the Louvre! #paris #art,2024-10-02 14:21:30
2,3,#history #architecture #notredame,Notre Dame,Notre Dame is as stunning as ever. #history #a...,2024-10-03 10:15:44
3,4,#paris #arcdetriomphe #adventure,Arc de Triomphe,Witnessing the grandeur of the Arc de Triomphe...,2024-10-04 09:45:00
4,5,#wanderlust #montmartre #travel,Montmartre,A perfect day spent wandering around Montmartr...,2024-10-05 17:20:18


In [21]:
df_naver_blog

,post_id,title,content,url,author,timestamp
0,101,Exploring Paris: A Travel Guide,"In this post, I'll take you through my recent ...",http://blog.naver.com/post101,author1,2024-09-15 10:12:45
1,102,Best Places to Visit in Bangkok,"Bangkok has so much to offer, from temples to ...",http://blog.naver.com/post102,author2,2024-09-16 14:23:32
2,103,A Day in Osaka: Food and Culture,Osaka is famous for its food and rich cultural...,http://blog.naver.com/post103,author3,2024-09-17 09:18:21
3,104,The Ultimate New York Travel Guide,"New York City is a place like no other, with i...",http://blog.naver.com/post104,author4,2024-09-18 16:45:00
4,105,Top Attractions in Paris,Paris is home to some of the world's most famo...,http://blog.naver.com/post105,author5,2024-09-19 11:30:10


In [22]:
df_places

,place_id,name,address,latitude,longitude,rating,reviews,place_type,opening_hours,photo_url
0,1,Eiffel Tower,"Champ de Mars, 5 Avenue Anatole France, 75007 ...",48.8584,2.2945,4.7,Amazing landmark with great views.,Tourist Attraction,09:00 AM - 12:00 AM,http://example.com/photo1.jpg
1,2,Louvre Museum,"Rue de Rivoli, 75001 Paris, France",48.8606,2.3376,4.8,"Incredible art collection, must-see!",Museum,09:00 AM - 06:00 PM,http://example.com/photo2.jpg
2,3,Notre Dame,"6 Parvis Notre-Dame - Pl. Jean-Paul II, 75004 ...",48.8530,2.3499,4.6,"A beautiful piece of history, very moving.",Church,07:45 AM - 06:45 PM,http://example.com/photo3.jpg
3,4,Arc de Triomphe,"Place Charles de Gaulle, 75008 Paris, France",48.8738,2.2950,4.7,"Breathtaking monument, iconic in every way.",Monument,10:00 AM - 11:00 PM,http://example.com/photo4.jpg
4,5,Montmartre,"Montmartre, 75018 Paris, France",48.8867,2.3431,4.5,"Charming area, full of artistic history.",Neighborhood,Open 24 hours,http://example.com/photo5.jpg


In [23]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import pinecone
from pinecone import Pinecone, ServerlessSpec

# Pinecone 초기화
pinecone = Pinecone(api_key="YOUR-API-KEY")

index = pinecone.Index('travel-index')  # 인덱스 이름 지정

# 임베딩 모델 로드
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [24]:
# 각 파일을 데이터프레임으로 읽기
df_instagram = pd.read_csv('instagram_posts_example.csv')
df_naver_blog = pd.read_csv('naver_blog_posts_example.csv')
df_places = pd.read_csv('places_data_example.csv')

# 도시 지정
city = 'paris'

# 인스타그램 데이터 임베딩 및 벡터 DB 저장
for i, row in df_instagram.iterrows():
    caption_embedding = model.encode(row['caption'])
    metadata = {
        "source": "instagram",
        "hashtag": row['hashtag'],
        "tagged_location": row['tagged_location']
    }
    index.upsert([{
        'id': f'instagram_{row["post_id"]}',
        'values': caption_embedding,
        'metadata': metadata
    }], namespace=city)

# 네이버 블로그 데이터 임베딩 및 벡터DB 저장
for i, row in df_naver_blog.iterrows():
    # 제목과 본문 임베딩
    title_embedding = model.encode(row['title'])
    content_embedding = model.encode(row['content'])

    # 제목과 본문 벡터를 결합
    combined_embedding = title_embedding + content_embedding

    # 메타데이터 설정
    metadata = {
        "source": "naver_blog",
        "title": row['title'],  # 제목도 메타데이터로 저장
    }
    index.upsert([{
        'id': f'naver_blog_{row["post_id"]}',
        'values': combined_embedding,  # 결합된 임베딩
        'metadata': metadata
    }], namespace=city)


# Google Places 데이터 임베딩 및 벡터 DB 저장
for i, row in df_places.iterrows():
    review_embedding = model.encode(row['reviews'])
    metadata = {
        "source": "google_places",
        "place_id": row['place_id'],
        "name": row['name'],
        "address": row['address'],
        "latitude": row['latitude'],
        "longitude": row['longitude'],
        "rating": row['rating'],
        "place_type": row['place_type'],
        "opening_hours": row['opening_hours'],
        "photo_url": row['photo_url']
    }
    index.upsert([{
        'id': f'google_places_{row["place_id"]}',
        'values': review_embedding,
        'metadata': metadata
    }], namespace=city)

print("데이터가 성공적으로 벡터 DB에 저장되었습니다.")

데이터가 성공적으로 벡터 DB에 저장되었습니다.


In [27]:
from pinecone import Pinecone, ServerlessSpec

# Pinecone 초기화
pinecone = Pinecone(api_key="YOUR-API-KEY")
index = pinecone.Index('travel-index')  # 인덱스 이름 지정

# 쿼리 벡터 생성
query = "Could you give me the information about Eiffel Tower?"
query_embedding = model.encode(query).tolist()

# 유사한 벡터 검색 (top_k=5로 상위 5개 결과)
search_results = index.query(vector=query_embedding, top_k=5, namespace='paris', include_metadata=True)


# 검색 결과 출력(유사도 점수 높은 순으로 출력)
for match in search_results['matches']:
    print(f"Vector ID: {match['id']}")
    print(f"Score: {match['score']}")
    print(f"Metadats: {match['metadata']}","\n")

Vector ID: instagram_1
Score: 0.597143769
Metadats: {'hashtag': '#travel #paris #explore', 'source': 'instagram', 'tagged_location': 'Eiffel Tower'} 

Vector ID: google_places_1
Score: 0.471007556
Metadats: {'address': 'Champ de Mars, 5 Avenue Anatole France, 75007 Paris, France', 'latitude': 48.8584, 'longitude': 2.2945, 'name': 'Eiffel Tower', 'opening_hours': '09:00 AM - 12:00 AM', 'photo_url': 'http://example.com/photo1.jpg', 'place_id': 1.0, 'place_type': 'Tourist Attraction', 'rating': 4.7, 'source': 'google_places'} 

Vector ID: instagram_2
Score: 0.417885482
Metadats: {'hashtag': '#art #louvre #museum', 'source': 'instagram', 'tagged_location': 'Louvre Museum'} 

Vector ID: naver_blog_105
Score: 0.411894083
Metadats: {'source': 'naver_blog', 'title': 'Top Attractions in Paris'} 

Vector ID: google_places_4
Score: 0.397509128
Metadats: {'address': 'Place Charles de Gaulle, 75008 Paris, France', 'latitude': 48.8738, 'longitude': 2.295, 'name': 'Arc de Triomphe', 'opening_hours': 

In [28]:
search_results

{'matches': [{'id': 'instagram_1',
              'metadata': {'hashtag': '#travel #paris #explore',
                           'source': 'instagram',
                           'tagged_location': 'Eiffel Tower'},
              'score': 0.597143769,
              'values': []},
             {'id': 'google_places_1',
              'metadata': {'address': 'Champ de Mars, 5 Avenue Anatole France, '
                                      '75007 Paris, France',
                           'latitude': 48.8584,
                           'longitude': 2.2945,
                           'name': 'Eiffel Tower',
                           'opening_hours': '09:00 AM - 12:00 AM',
                           'photo_url': 'http://example.com/photo1.jpg',
                           'place_id': 1.0,
                           'place_type': 'Tourist Attraction',
                           'rating': 4.7,
                           'source': 'google_places'},
              'score': 0.471007556,
              